In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
max_long = 25

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [5]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.2 MB/s 


In [6]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [7]:
import token_glove
token_glove.create_list, token_glove.token_ize, token_glove.voc_token, token_glove.dict_token, token_glove.token_tweet, token_glove.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [8]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [13]:
df = data.get_data(addon=True)
cleaned_df = data.clean_data(df)

In [14]:
cleaned_df

,id,text,target
0,1,deed reason earthquake may allah forgive u,1
1,4,forest fire near la canada,1
2,5,resident asked shelter place notified officer ...,1
3,6,people receive wildfire evacuation order calif...,1
4,7,got sent photo alaska smoke wildfire school,1
...,...,...,...
11365,11365,medium warned u well advance wrecked whole nig...,0
11366,11366,feel directly attacked consider moonbin amp ji...,0
11367,11367,feel directly attacked consider moonbin amp ji...,0
11368,11368,ok remember outcast nd au au wrecked nerve nd ...,0


In [15]:
text = token_glove.create_list(cleaned_df['text'])
token = token_glove.token_ize(text)
X = token_glove.token_tweet(text, token)

In [16]:
word_vector_matrix = token_glove.glove_vector(token)

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [23]:
y = cleaned_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.3, stratify = y)

In [24]:
model5 = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model5.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model5.add(LSTM(128,return_sequences = True))
model5.add(Dropout(0.5))
model5.add(LSTM(32, return_sequences = False))
model5.add(Dropout(0.4))
model5.add(Dense(32, activation = 'relu'))
model5.add(Dropout(0.2))
model5.add(Dense(1, activation = 'sigmoid'))
model5.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=[metrics.f1_m])
history3 = model5.fit(X_train, y_train, batch_size=16, epochs = 400, validation_data = (X_test, y_test), callbacks = [es])

Epoch 1/400
830/830 [==============================] - 78s 87ms/step - loss: 0.4673 - f1_m: 0.5304 - val_loss: 0.4089 - val_f1_m: 0.6294
Epoch 2/400
830/830 [==============================] - 72s 86ms/step - loss: 0.3939 - f1_m: 0.6688 - val_loss: 0.3990 - val_f1_m: 0.6345
Epoch 3/400
830/830 [==============================] - 70s 84ms/step - loss: 0.3602 - f1_m: 0.6835 - val_loss: 0.4022 - val_f1_m: 0.6516
Epoch 4/400
830/830 [==============================] - 69s 83ms/step - loss: 0.3277 - f1_m: 0.7171 - val_loss: 0.3915 - val_f1_m: 0.6269
Epoch 5/400
830/830 [==============================] - 69s 83ms/step - loss: 0.2978 - f1_m: 0.7516 - val_loss: 0.4056 - val_f1_m: 0.6366
Epoch 6/400
830/830 [==============================] - 70s 85ms/step - loss: 0.2679 - f1_m: 0.7713 - val_loss: 0.4341 - val_f1_m: 0.6401
Epoch 7/400
830/830 [==============================] - 72s 87ms/step - loss: 0.2382 - f1_m: 0.8047 - val_loss: 0.4579 - val_f1_m: 0.6484
Epoch 8/400
830/830 [====================